# Pair-wise image order Learning with Siamese Networks

This is the jupyter notebook that accompanies

## Imports
All the imports are defined here

In [ ]:
%matplotlib inline
import os
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import random
from PIL import Image
import torch
from torch.autograd import Variable
import PIL.ImageOps    
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import logging
from tqdm import tqdm
import time

## Helper functions
Set of helper functions

In [ ]:
def imshow(img,text=None,should_save=False):
    npimg = img.numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()    

def show_plot(iteration,loss):
    plt.plot(iteration,loss)
    plt.show()

## Configuration Class
A simple class to manage configuration

In [ ]:
class Config():
    imageFiles_path = {}
    imageFiles_path['train'] = "../shared_data/train_images/"
    imageFiles_path['test'] =  "../shared_data/val_images/"
    #train/test split info
    movie_names = {}        
    movie_names['train'] = np.load("../shared_data/train_vids.npy")
    movie_names['test'] =  np.load("../shared_data/test_vids.npy")
    
    train_batch_size = 32
    test_batch_size = 32
    
def Extract_splitFiles(files,split,movie_names):
    split_image_files = []
    split_movie_names = movie_names
    for fname in files:
        #fname: ETqlg-6bflc_3600_4125
        vname = fname[0:11]
        if vname in split_movie_names:
            split_image_files.append(fname)
    return split_image_files

## Custom Dataset Class
This dataset generates a pair of images. 0 for geniune pair and 1 for imposter pair

In [ ]:
'''
input:
- video2cnames = {'-9GYpCvGIgM':[clip_name1, clip_names], ……}
- gap: the interval step num between image pairs
output:
- data: save img pairs = [img1_path, img2_path, order]
        among which, img1 = {'clip_name':……,'img_name': ……,'img_tensor_path':……}
'''
def get_train_pairs_gap_n(video2clips,path,gap):
    data = []
    for key in video2clips:
        clip_names = video2clips[key]
        cnum = len(clip_names )
        start_frames = np.array([int(s.split('_')[-2]) for s in clip_names])
        index = start_frames.argsort() 
        for i in range(cnum - gap):
            img1_file = clip_names[index[i]]
            img2_file = clip_names[index[i+gap]]
            img1_names = sorted(os.listdir(path+img1_file))
            img2_names = sorted(os.listdir(path+img2_file))
           
            leng = len(img1_names)
         
            for k in range(leng-1,leng):#each clip only use one image
                order = 0
                name1 = img1_names[k]
                name2 = img2_names[k]
                #name2 = random.choice(img2_names)
                img1_path = img1_file + '/' + name1
                img2_path = img2_file + '/' + name2
                #we need to make sure approx 50% of images order is 1 (image i > image j)
                #the rest 50% of images order is 0(image i < image j)
                should_get_first_order = random.randint(0,1) 
                if should_get_first_order == 1:
                    order = 1
                    tmp = img1_path
                    img1_path = img2_path
                    img2_path = tmp
                data.append([img1_path,img2_path,order])
    return data 

def get_train_pairs_gap_more_than_n(video2clips,path,gap):
    data = []
    for key in video2clips:
        clip_names = video2clips[key]
        cnum = len(clip_names )
        start_frames = np.array([int(s.split('_')[-2]) for s in clip_names])
        index = start_frames.argsort() 
        for i in range(cnum - gap):
            img1_file = clip_names[index[i]]
            img2_file = []
            for j in range(gap,len(index)-i):
                img2_file = clip_names[index[i+j]]
                img1_names = sorted(os.listdir(path+img1_file))
                img2_names = sorted(os.listdir(path+img2_file))
                
                leng = len(img1_names)
                
                for k in range(leng-1,leng):#each clip only use one image
                    order = 0
                    name1 = img1_names[k]
                    name2 = img2_names[k]
                    img1_path = img1_file + '/' + name1
                    img2_path = img2_file + '/' + name2
                    #we need to make sure approx 50% of images order is 1 (image i > image j)
                    #the rest 50% of images order is 0(image i < image j)
                    should_get_first_order = random.randint(0,1) 
                    if should_get_first_order == 1:
                        order = 1
                        tmp = img1_path
                        img1_path = img2_path
                        img2_path = tmp
                    data.append([img1_path,img2_path,order])
    return data

In [ ]:
class SiameseNetworkDataset(torch.utils.data.Dataset):
    
    def __init__(self,path,split,gap,transform=None):
        super(SiameseNetworkDataset, self).__init__()  
        self.path = path
        self.split = split
        image_files = os.listdir(self.path)
        self.movie_names = Config.movie_names[split]
        self.split_image_files = Extract_splitFiles(image_files,self.split,self.movie_names)
        self.clip_num = len(self.split_image_files)
        self.pairs = []
        self.video2clips = {}
        print(self.split)
        for vname in self.movie_names:
            self.video2clips[vname] = []
        
        for cname in self.split_image_files:
            vname = cname[0:11]
            self.video2clips[vname].append(cname)
        
        
        for key in self.video2clips:
            if len(self.video2clips[key]) < 2:
                #print(key," only has {} steps".format(len(self.video2clips[key])))
                self.split_image_files.remove(self.video2clips[key][0])
        
        self.gap = gap
        #print "gap = {}".format(gap)
        for g in gap:
            if g < 5:
                pairs = get_train_pairs_gap_n(self.video2clips,path,g)
                self.pairs.extend(pairs)
            else:
                pairs = get_train_pairs_gap_more_than_n(self.video2clips,path,g)
                self.pairs.extend(pairs)
        random.shuffle(self.pairs)
        
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self,index):
        img1_path,img2_path,order = self.pairs[index]
        img0 = torch.load(self.path + img1_path)
        img1 = torch.load(self.path + img2_path) 
        return img0, img1, int(order)
        

## Using Image Folder Dataset

In [ ]:
gap = [1,2,3,4,5]
siamese_trainDataset = SiameseNetworkDataset(
        path=Config.imageFiles_path['train'],
        split='train',
        gap = gap
)

siamese_testDataset  = SiameseNetworkDataset(
        path=Config.imageFiles_path['test'],
        split='test',
        gap = gap
)
print"trainset data size:",len(siamese_trainDataset)
print"testset data size:",len(siamese_testDataset)

# Visualising some of the data

In [ ]:
vsiamese_testDataset  = SiameseNetworkDataset(
        path=Config.imageFiles_path['test'],
        split='test',
        gap = gap
)

vis_dataloader = DataLoader(vsiamese_testDataset,
                        shuffle=True,
                        num_workers=0,
                        batch_size=8)
dataiter = iter(vis_dataloader)


example_batch = next(dataiter)
concatenated = torch.cat((example_batch[0],example_batch[1]),0)
imshow(torchvision.utils.make_grid(concatenated))
print(example_batch[2].numpy())
#for i in zip(example_batch[3],example_batch[4]):
#    print(i)

## Neural Net Definition
We will use a standard convolutional neural network

In [ ]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        resnet18 = torchvision.models.resnet18(pretrained = True)
        self.cnn1 = resnet18
        num_ftrs = resnet18.fc.out_features
        
        self.fc1 = nn.Sequential(
            nn.Linear(num_ftrs*2, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 2)
        )

    def forward_once(self, x):
        output = self.cnn1(x)   
        output = output.view(output.size()[0], -1)

        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        #concate output1&output2
        output = torch.cat((output1, output2), dim=1)
        output = self.fc1(output)
        
        return output

## Training Time!

In [ ]:
def save_checkpoint(state, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    logging.info('Save checkpoint fo {}'.format(filename))
    
def mkdir(d):
    if not os.path.isdir(d) and not os.path.exists(d):
        os.system('mkdir -p {}'.format(d))

In [ ]:
log_file = "./contrastive_loss/"
snapshot = "./checkpoints/"
mkdir(snapshot)
mkdir(log_file)

In [ ]:
train_dataloader = DataLoader(siamese_trainDataset,
                        shuffle=True,
                        num_workers=0,
                        pin_memory = True,
                        batch_size=Config.train_batch_size)
test_dataloader = DataLoader(siamese_testDataset,
                             num_workers=0,
                             shuffle=True,
                             pin_memory = True,
                             batch_size=Config.test_batch_size)

In [ ]:
net = SiameseNetwork().cuda()
#net = SiameseNetwork()
criterion = nn.CrossEntropyLoss()
base_lr = 0.0001
step_size = 5
Config.train_number_epochs = 20
optimizer = optim.SGD(net.parameters(), lr=base_lr, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=0.1)

In [ ]:
counter = []
loss_history = [] 
iteration_number = 0
init_epoch = 0

In [ ]:
lr = base_lr
lr_iter = 0
for epoch in range(init_epoch,Config.train_number_epochs):
    scheduler.step()
    lr_iter += 1
    #train for one epoch-----------------------------------------------------------------------------------
    epoch_loss = 0.0
    epoch_acc = 0.0
    i = 0
    with tqdm(train_dataloader, position=0, leave=True) as t:
        for data in t:
            img0, img1 , label = data
            img0, img1 , label = img0.cuda(), img1.cuda(), label.cuda()
            optimizer.zero_grad()
        
            output= net(img0,img1)
            loss_contrastive = criterion(output,label)
        
            epoch_loss += loss_contrastive[0].item()
            pred = torch.max(output, 1)[1]
            epoch_correct = (pred == label).sum()
            epoch_acc += epoch_correct[0].item()
        
            loss_contrastive.backward()
            optimizer.step()
            iters = len(siamese_trainDataset)/Config.train_batch_size
            if i % int(iters/10) == 1 :
                iteration_number +=iters
                counter.append(iteration_number)
                loss_history.append(loss_contrastive.item())
            i += 1  
        if lr_iter%step_size == 0:
            lr = lr*0.1
    t.close()    
    print('Epoch: {}, Train Loss: {:.6f}, Train Acc: {:.6f}'.format(epoch, epoch_loss / (len(siamese_trainDataset)), epoch_acc / (len(siamese_trainDataset))))     
    
    #evaluate for every epoch----------------------------------------------------------------------------------
    net.eval()
    eval_acc = 0.0
    eval_loss = 0.0
    i = 0
    with tqdm(test_dataloader, position=0, leave=True) as test:
        for data in test:
            img0, img1, label = data
            img0, img1, label = img0.cuda(), img1.cuda(), label.cuda()
            out = net(img0,img1)
            loss_contrastive = criterion(out,label)
            
            pred = torch.max(out, 1)[1]
            num_correct = (pred == label).sum()
            eval_acc += num_correct[0].item()
            eval_loss += loss_contrastive[0].item()
            i += 1
    test.close()
    print('Epoch: {}, Test Loss: {:.6f}, Test Acc: {:.6f}'.format(epoch,eval_loss/len(siamese_testDataset),eval_acc/len(siamese_testDataset)))
    #write to logfile
    logfile=open(log_file + 'contrastive_gap{}.txt'.format(gap),'a')
    logfile.write( 'Epoch: {},lr: {:.6f}, Train Loss: {:.6f}, Train Acc: {:.6f}, Test Loss: {:.6f}, Test Acc: {:.6f}\n'.format(
        epoch, lr, epoch_loss / len(siamese_trainDataset), epoch_acc / len(siamese_trainDataset),
        eval_loss/len(siamese_testDataset),eval_acc/len(siamese_testDataset)))
    logfile.close()
    
    #save model paramers for each epoch
    filename = snapshot+'checkpoint_gap{}_epoch{}.pth.tar'.format(gap,epoch)
    save_checkpoint(
        {
            'epoch':epoch,
            'lr':lr,
            'gap':gap,
            'state_dict':net.state_dict()
        },
        filename
    )
#show_plot(counter,loss_history)

## load model & model evaluation


In [ ]:
from tqdm import tnrange, tqdm_notebook
def Evaluation(test_dataloader,model,siamese_testDataset):
    #model evaluation
    model.eval()
    eval_acc = 0.0
    for data in tqdm(test_dataloader):
        img0, img1, label = data
        img0, img1, label = img0.cuda(), img1.cuda(), label.cuda()
        torch.no_grad()
        out = model(img0,img1)
        pred = torch.max(out, 1)[1]
        num_correct = (pred == label).sum()
        eval_acc += num_correct[0].item()
    eval_acc =  eval_acc/len(siamese_testDataset)
    print('epoch:{}  test acc: {:.6f}'.format(epoch,eval_acc))

In [ ]:
#load trained model
model = SiameseNetwork().cuda()

checkpoint_path = '' #Your checkpoint path
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['state_dict'])
epoch = checkpoint['epoch']
siamese_testDataset  = SiameseNetworkDataset(
        path=Config.imageFiles_path['test'],
        split='test',
        gap = [1,2,3,4,5]
)
print(len(siamese_testDataset))
test_dataloader = DataLoader(siamese_testDataset,num_workers=0,shuffle=True,pin_memory = True,batch_size=Config.test_batch_size)
Evaluation(test_dataloader,model,siamese_testDataset)

# Calculate choice accuracy on image ordering task 

In [ ]:
def Get_Validation_Info(json_path):
    qinfos = []
    with open(json_path, "r") as file:
        for index,f in enumerate(file):
            line  = json.loads(f)
            question_info = {}
            
            question_info['question_id'] = line['question_id']# number: 1 (related image dir name: 1/)
            question_info['video_id']  = line['video_id']#"-9GYpCvGIgM"
            question_info['step_caption'] = line['step_caption']#list of step captions
            question_info['groundtruth'] = line['groundtruth'] #[4, 1, 2, 5, 3]
            question_info['candidate_answer'] = line['candidate_answer']
            qinfos.append(question_info)
    return qinfos  

def Get_QueryImages(imgs_path):
    imgs = []
    transform=torchvision.transforms.Compose([
            torchvision.transforms.Resize(224),
            torchvision.transforms.CenterCrop(224),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                             [0.229, 0.224, 0.225])
        ])
    img_name = sorted(os.listdir(imgs_path))
    #print(img_name)
    for n in img_name:
        if n[-4:] != '.jpg':
            continue
        img_path = imgs_path + n
        #read image
        with open(img_path, 'rb') as f:
            img = PIL.Image.open(f)
            img = img.convert('RGB') 
            img = transform(img)
            imgs.append(img)
    return imgs

def if_same(select_answer,ground_truth):
    count = 0
    for i in range(len(select_answer)):
        count += (select_answer[i] == ground_truth[i])
    return int(count == 5)

In [ ]:
from scipy.stats import pearsonr
import json
from Levenshtein import *

split = "dev" #"test"
#dev
'''
path = "/data3/YouMakeup/data/task/image_ordering/dev/"
validation_path = path + "image_ordering_validation.json"
images_path = path + "images/"
'''
#test

path = "/data3/YouMakeup/data/task/image_ordering/test/"
validation_path = path + "image_ordering.json"
images_path = path + "images/"

questions_info = Get_Validation_Info(validation_path)
select_correct_new_confidence = 0.0
select_correct_dis = 0.0
completion_correct = 0.0
init_pic_acc = 0.0
init_pic = []
init_gdth_pic = []
init_pic_acc_confidence = 0.0
init_pic_confidence = []
select_answer = []
#save predict answer
final_answer = {}
#load model
model = SiameseNetwork().cuda()

checkpoint_path =  '' #Your chenckpoint path
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['state_dict'])
model.eval()
for i,query in enumerate(questions_info):
    #get five query imgs]
    img_dir = str(query['question_id'])
    imgs_path = images_path + img_dir + '/'
    imgs = Get_QueryImages(imgs_path)
    #12，13，14，15，23，24，25，34，35，45 pairs comparasion
    imgs_list1 = [imgs[0],imgs[0],imgs[0],imgs[0],imgs[1],imgs[1],imgs[1],imgs[2],imgs[2],imgs[3]]
    imgs_list2 = [imgs[1],imgs[2],imgs[3],imgs[4],imgs[2],imgs[3],imgs[4],imgs[3],imgs[4],imgs[4]]
    imgs_list1 = torch.stack(imgs_list1).float()
    imgs_list1 = torch.autograd.Variable(imgs_list1).cuda()
    imgs_list2 = torch.stack(imgs_list2).float()
    imgs_list2 = torch.autograd.Variable(imgs_list2).cuda()
    pred = None
    predc = None
    with torch.no_grad():
        out = F.softmax(model(imgs_list1,imgs_list2))
        pred = np.array(torch.max(out, 1)[1])
        predc = np.array(out[:,1])
 
    ground_truth = query['groundtruth']
    print("------query {}  ground truth {}-------".format(i+1,ground_truth ))
    completion_answer = []
    score1 = pred[0]+pred[1]+pred[2]+pred[3]
    score2 = pred[4]+pred[5]+pred[6]+(1-pred[0])
    score3 = pred[7]+pred[8]+(1-pred[1])+(1-pred[4])
    score4 = pred[9]+(1-pred[2])+(1-pred[5])+(1-pred[7])
    score5 = (1-pred[3])+(1-pred[6])+(1-pred[8])+(1-pred[9])
    order = np.argsort([score1,score2,score3,score4,score5])
    order = list(order+1)
    completion_answer = order
    count = if_same(completion_answer,ground_truth)
  
    
    #choose answer algorithm
    pred_dict = {'12':predc[0],'13':predc[1],'14':predc[2],'15':predc[3],'23':predc[4],'24':predc[5],'25':predc[6],'34':predc[7],'35':predc[8],'45':predc[9],
                 '21':1-predc[0],'31':1-predc[1],'41':1-predc[2],'51':1-predc[3],'32':1-predc[4],'42':1-predc[5],'52':1-predc[6],'43':1-predc[7],'53':1-predc[8],'54':1-predc[9]}
    max_score = -1
    confidence = {}
    for answer in query['candidate_answer']:
        score = 0
        pairs = []
        for j in range(4):
            for k in range(j+1,5):   
                pairs.append(str(answer[k])+str(answer[j]))
        for p in pairs:
            score += pred_dict[p]
        confidence[str(answer)] = score
        if score > max_score:
            max_score = score
            select_answer = answer
    count = if_same(select_answer,ground_truth)
    select_correct_new_confidence += count
    print "choose answer:",select_answer, bool(count)
    final_answer[query['question_id']] = select_answer
    init_pic_acc += int(ground_truth[0] == select_answer[0])
    init_pic.append(select_answer[0])
    init_gdth_pic.append(select_answer[0])
    
    #calculate init image
    pick_init_pic = -1
    max_prob = -1
    for img_id in ['1','2','3','4','5']:
        prob = 0.0
        for rest_id in ['1','2','3','4','5']:
            if rest_id != img_id:
                prob += pred_dict[rest_id+img_id]
               # prob += int(pred_dict[img_id+rest_id]>0.5)
        prob /= 4.0
        #print(prob)
        if prob > max_prob:
            max_prob = prob
            pick_init_pic = int(img_id)
    init_pic_acc_confidence += int(ground_truth[0] == pick_init_pic)
    init_pic_confidence.append(pick_init_pic)
    print("init image",pick_init_pic,"ground truth",ground_truth[0],bool(ground_truth[0] == pick_init_pic))
    


In [ ]:
acc = init_pic_acc_confidence/len(questions_info)
np.save("init_pics_{}_{:.3f}.npy".format(split,acc),init_pic_confidence)
print("init_pics_{}_{:.3f}.npy".format(split,acc))
choice_acc = select_correct_new_confidence/len(questions_info
print("Choice accuracy: {:.3f}".format(select_correct_new_confidence/len(questions_info)))

In [ ]:
path = "pairwise_{}_answer{:.3f}.json".format(split,choice_acc)
json_data_save(path, final_answer)